In [54]:
%matplotlib inline
import json
import glob
import pandas as pd
import urllib2
import time
import re

Need to rerun nyt api (gather_data)

In [46]:
with open('../output/article_search.json') as f:
    	dat = json.load(f)

files = glob.glob('../output/*facebook_dat.csv')
outdat = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    outdat = pd.concat([outdat, d])


In [39]:
outdat.head()

,id,link,reactions,shares,can_comment,comments
0,5281959998_10150991495809999,https://www.nytimes.com/2016/12/01/upshot/trum...,2067.0,597.0,False,125.0
0,5281959998_10150989221999999,http://www.nytimes.com/2016/12/01/world/austra...,669.0,94.0,False,32.0
0,5281959998_10150988663124999,http://www.nytimes.com/2016/12/01/world/europe...,200.0,27.0,False,31.0
1,5281959998_10150988419179999,http://www.nytimes.com/2016/12/01/us/politics/...,2149.0,398.0,False,489.0
2,5281959998_10150988331464999,http://www.nytimes.com/2016/12/01/opinion/dona...,8153.0,2892.0,False,409.0


In [95]:
import json
import pandas as pd
import numpy as np

def core_url(url):
    m = re.search('(http(s)?://(((www|.+\.blogs)\.nytimes\.com.+)(roomfordebate/.+|/$|.html)))', url)
    if m:
        return(m.group(3))
    else:
        return('********url %s not pulled' % url)

def get_keywords(dat):
    """
    pull out data from json object (dat) for keywords analysis
    """
    out = []
    for art in dat:
    	if art.get('keywords'):
	        for child in art.get('keywords'):
	            out.append([art['_id'], art['section_name'], child['value'], child['name'],])
        else:
	    	out.append([art['_id'], art['section_name'], np.nan, np.nan])
    
    df = pd.DataFrame(out, columns = ['id', 'section', 'keyword', 'entity_type'])
    return(df)

def count_keywords(df):
    """
    count keywords by entity type
    """
    df['keyword_count'] = df.groupby(['entity_type', 'keyword'])['id'].transform('count')
    df['section_count'] = df.groupby(['section'])['id'].transform('nunique')
    return(df)

def compute_influence(nyt_dat, fb_dat):
    out = []
    for art in nyt_dat:
        out.append([art['pub_date'], art['comments'], art['web_url'], art['_id'],])
    
    df = pd.DataFrame(out, columns = ['pub_date', 'nyt_comments', 'web_url', 'id'])
    
    df['clean_url'] = df.web_url.apply(core_url)
    fb_dat['clean_url'] = fb_dat.link.apply(core_url)
    df = pd.merge(df.loc[:,['pub_date', 'nyt_comments', 'clean_url']], 
                  fb_dat.loc[:,['reactions', 'shares', 'comments', 'clean_url']], 
                  how='left', on='clean_url')
    df.nyt_comments.replace(0, np.nan, inplace=True)
    df['nyt_comments'] = (df.nyt_comments-df.nyt_comments.mean())/df.nyt_comments.std()
    df['reactions'] = (df.reactions-df.reactions.mean())/df.reactions.std()
    df['shares'] = (df.shares-df.shares.mean())/df.shares.std()
    df['comments'] = (df.comments-df.comments.mean())/df.comments.std()
    col_list = ['nyt_comments', 'reactions', 'shares', 'comments']
    df['influence'] = df[col_list].mean(axis=1)
    
    return(df)


In [ ]:
def main():
    with open('output/article_search.json') as f:
    	dat = json.load(f)
	
	df = get_keywords(dat)
	df = count_keywords(df)
    files = glob.glob('../output/*facebook_dat.csv')
    fb_dat = pd.DataFrame()
    for f in files:
        d = pd.read_csv(f)
        fb_dat = pd.concat([fb_dat, d])
    df_infl = compute_influence(dat, fb_dat)
    df.to_csv('output/counts.csv', index=False, encoding='utf-8')
    df_infl.to_csv('output/infl.csv', index=False, encoding='utf-8')

#if __name__=='__main__':

#    main()

In [96]:
with open('../output/article_search.json') as f:
    dat = json.load(f)
    
files = glob.glob('../output/*facebook_dat.csv')
fb_dat = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    fb_dat = pd.concat([fb_dat, d])

df_infl = compute_influence(dat, fb_dat)
df_infl.head()

,pub_date,nyt_comments,clean_url,reactions,shares,comments,influence
0,2016-11-08T16:10:11Z,-0.816670,dotearth.blogs.nytimes.com/2016/11/08/an-anthr...,NaN,NaN,NaN,-0.816670
1,2016-11-10T15:39:35Z,0.092837,www.nytimes.com/2016/11/11/us/politics/donald-...,-0.174971,-0.190004,-0.420182,-0.173080
2,2016-11-30T08:21:24+0000,-0.587909,www.nytimes.com/2016/11/30/opinion/cities-and-...,NaN,NaN,NaN,-0.587909
3,2016-11-23T17:44:55+0000,NaN,www.nytimes.com/2016/11/23/us/politics/mike-bl...,2.397546,0.014984,-0.190145,0.740795
4,2016-11-18T09:04:40+0000,-0.759480,www.nytimes.com/2016/11/19/world/asia/china-tr...,0.099130,0.020467,-0.306989,-0.236718


In [122]:
df = get_keywords(dat)
df = count_keywords(df)
files = glob.glob('../output/*facebook_dat.csv')
fb_dat = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    fb_dat = pd.concat([fb_dat, d])


In [123]:
def get_page(app_id, app_secret):
    baseurl = 'https://graph.facebook.com/v2.8/nytimes/feed/'
    access_token = '&access_token=' + app_id + '|' + app_secret
    fields = '?fields=id,name,link,created_time,reactions.summary(true),comments.summary(true),shares&limit=25'
    url = baseurl+fields+access_token
    dat = json.loads(request_until_succeed(url))
    
    return(dat)

def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)

    return response.read()

def page_through(app_id, app_secret):

    has_next_page = True
    in_date_range = True
    with open('../output/article_search.json') as f:
        nyt_dat = json.load(f)

    nyt_urls = []
    for i in nyt_dat:
        nyt_urls.append(core_url(i['web_url']))

    items = get_page(app_id, app_secret)
    print('queried to date %s' % items['data'][24]['created_time'])

    process_items(items, nyt_urls)

    while has_next_page & in_date_range:
        if 'paging' not in items.keys():
            has_next_page=False

        if items['data'][0]['created_time'][0:7]=='2016-10':
            in_date_range = False

        items = json.loads(request_until_succeed(items['paging']['next']))
        process_items(items, nyt_urls)

def process_items(json_dat, target_urls):
    print('********Writing file for posts at %s' % json_dat['data'][0]['created_time'])
    with open('%s_facebook_dat.csv' % datetime.datetime.now(), 'wb') as outfile:
        w = csv.writer(outfile)
        w.writerow(['id', 'link', 'reactions', 'shares', 'can_comment', 'comments'])
        earliest_date = datetime.datetime.strptime('2016-11-01', '%Y-%m-%d')
        for item in json_dat['data']:
            try:
                if core_url(item['link']) in target_urls:
                    w.writerow([item['id'],
                               item['link'],
                               item['reactions']['summary']['total_count'],
                               item['shares']['count'],
                               item['comments']['summary']['can_comment'],
                               item['comments']['summary']['total_count']])
            except:
                pass
                        
def core_url(url):
    m = re.search('(http(s)?://(((www|.+\.blogs)\.nytimes\.com.+)(roomfordebate/.+|/$|.html)))', url)
    if m:
        return(m.group(3))
    else:
        return('********url %s not pulled' % url)

In [193]:
with open('../output/article_search.json') as f:
        nyt_dat = json.load(f)

nyt_urls = []
for i in nyt_dat:
    nyt_urls.append(core_url(i['web_url']))
    print(core_url(i['web_url']))

dotearth.blogs.nytimes.com/2016/11/08/an-anthropocene-journey/
www.nytimes.com/2016/11/11/us/politics/donald-trump-climate-change.html
www.nytimes.com/2016/11/30/opinion/cities-and-states-lead-on-climate-change.html
www.nytimes.com/2016/11/23/us/politics/mike-bloomberg-donald-trump-climate-change.html
www.nytimes.com/2016/11/19/world/asia/china-trump-climate-change.html
www.nytimes.com/2016/11/16/opinion/donald-trump-help-heal-the-planets-climate-change-problem.html
www.nytimes.com/2016/11/17/science/paris-accord-global-warming-iea.html
www.nytimes.com/2016/11/29/business/economy/trumps-climate-policies-may-work-out-in-the-planets-favor.html
www.nytimes.com/2016/11/16/world/united-nations-climate-change-trump.html
www.nytimes.com/2016/11/25/opinion/climate-deniers.html
www.nytimes.com/2016/11/15/opinion/trump-has-declared-climate-war-but-my-generation-will-win.html
www.nytimes.com/2016/11/03/business/energy-environment/united-nations-cop22-summit-pollution-monitoring.html
www.nytimes.c

In [205]:
app_id = 'XXX'
app_secret = 'XXX'
dat = page_through(app_id, app_secret)

queried to date 2016-12-30T13:35:00+0000
********Writing file for posts at 2016-12-30T23:00:00+0000
********Writing file for posts at 2016-12-30T13:05:00+0000
********Writing file for posts at 2016-12-29T22:25:00+0000
********Writing file for posts at 2016-12-29T12:36:44+0000
********Writing file for posts at 2016-12-28T20:28:24+0000
********Writing file for posts at 2016-12-28T06:40:00+0000
********Writing file for posts at 2016-12-27T19:25:01+0000
********Writing file for posts at 2016-12-27T05:55:00+0000
********Writing file for posts at 2016-12-26T15:55:00+0000
********Writing file for posts at 2016-12-25T19:25:00+0000
********Writing file for posts at 2016-12-24T21:55:00+0000
********Writing file for posts at 2016-12-24T06:55:00+0000
********Writing file for posts at 2016-12-23T16:55:00+0000
********Writing file for posts at 2016-12-23T06:55:00+0000
********Writing file for posts at 2016-12-22T20:25:00+0000
********Writing file for posts at 2016-12-22T11:55:32+0000
********Writing

In [48]:
with open('../output/article_search.json') as f:
    	nyt_dat = json.load(f)

In [58]:
for i in nyt_dat:
    print i['web_url']

http://dotearth.blogs.nytimes.com/2016/11/08/an-anthropocene-journey/
http://www.nytimes.com/2016/11/11/us/politics/donald-trump-climate-change.html
http://www.nytimes.com/2016/11/30/opinion/cities-and-states-lead-on-climate-change.html
http://www.nytimes.com/2016/11/23/us/politics/mike-bloomberg-donald-trump-climate-change.html
http://www.nytimes.com/2016/11/19/world/asia/china-trump-climate-change.html
http://www.nytimes.com/2016/11/16/opinion/donald-trump-help-heal-the-planets-climate-change-problem.html
http://www.nytimes.com/2016/11/17/science/paris-accord-global-warming-iea.html
http://www.nytimes.com/2016/11/29/business/economy/trumps-climate-policies-may-work-out-in-the-planets-favor.html
http://www.nytimes.com/2016/11/16/world/united-nations-climate-change-trump.html
http://www.nytimes.com/2016/11/25/opinion/climate-deniers.html
http://www.nytimes.com/2016/11/15/opinion/trump-has-declared-climate-war-but-my-generation-will-win.html
http://www.nytimes.com/2016/11/03/business/en

In [5]:
with open('../output/article_search.json') as f:
    	dat = json.load(f)

#comments=[]
#for i, j in enumerate(dat):
#    time.sleep(0.1)
#    print('getting comment count for article {} of {}'.format(i, len(dat)))
#    comments.append(get_comment_count('4d8f2843a565922de562726f9d86a63e:1:60193026', j['web_url']))

In [34]:
def get_comment_count(api_key, url):
    """
    Get one page of article data. 
    Feed in your api key, your query term, the dates to filter by, and the page number.

    query is formatted as 'foo+bar'

    dates are formatted YYYYMMDD
    
    returns json formatted data.
    """
    
    baseurl = 'http://api.nytimes.com/svc/community/v3/user-content/url.json'
    req = baseurl + '?api-key=' + api_key + '&url=' + url    
    resp = urllib2.urlopen(req)
    dat = json.loads(resp.read())
    time.sleep(0.1)
    
    return(dat['results']['totalCommentsFound'])

In [37]:
import json
import pandas as pd
import numpy as np

def get_keywords(dat):
    """
    pull out data from json object (dat) for keywords analysis
    """
    out = []
    for art in dat:
    	if art.get('keywords'):
	        for child in art.get('keywords'):
	            out.append([art['_id'], art['section_name'], child['value'], child['name'],])
        else:
	    	out.append([art['_id'], art['section_name'], np.nan, np.nan])
    
    df = pd.DataFrame(out, columns = ['id', 'section', 'keyword', 'entity_type'])
    return(df)

def count_keywords(df):
    """
    count keywords by entity type
    """
    df['keyword_count'] = df.groupby(['entity_type', 'keyword'])['id'].transform('count')
    df['section_count'] = df.groupby(['section'])['id'].transform('nunique')
    return(df)

def main():
    with open('output/article_search.json') as f:
    	dat = json.load(f)
	
	df = get_keywords(dat)
	df = count_keywords(df)
	df.to_csv('output/counts.csv', index=False, encoding='utf-8')

if __name__=='__main__':
    main()

17
510
141
0
48
464
0
0
0
0


In [32]:
dat[1][u'comments'] = 0

## Resources
http://minimaxir.com/2015/07/facebook-scraper/